# Prepare model files

In [1]:
import os
import glob
import json
from os.path import join as opj
import numpy as np
import pandas as pd

Project directories:

In [2]:
scratch_dir = os.environ['PI_SCRATCH']
data_dir = opj(scratch_dir, 'OPUS', 'BIDS_data')
model_dir = opj(data_dir, 'derivatives', 'l1model')

if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
    
print('Reading data from: %s' % data_dir)
print('Placing model specification in: %s' % model_dir)

Reading data from: /scratch/groups/hyo/OPUS/BIDS_data
Placing model specification in: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model


Find subjects:

In [3]:
subjects = [os.path.basename(x) for x in glob.glob(opj(data_dir, 'sub-*'))]
print('Found subjects: %s' % subjects)

Found subjects: ['sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08']


Helper functions:

In [4]:
mean_center = lambda x: x - np.nanmean(x)

## Self task

Contrasts:

In [5]:
self_names = ['self', 'motor-left', 'motor-right', 'self-chosen', 'self-unchosen']
self_con = [
    ('self: chosen', 'T', self_names, [0, 0, 0, 1, 0]),
    ('self:  unchosen', 'T', self_names, [0, 0, 0, 0, 1]),
    ('self: chosen > unchosen', 'T', self_names, [0, 0, 0, 1, -1]),
    ('self: unchosen > chosen', 'T', self_names, [0, 0, 0, -1, 1]),
    ('left > right', 'T', self_names, [0, 1, -1, 0, 0]),
    ('right > left', 'T', self_names, [0, -1, 1, 0, 0]),
    ('task', 'T', self_names, [1, 0, 0, 0, 0]),
    ('rest', 'T', self_names, [-1, 0, 0, 0, 0])
]

self_con_out = opj(model_dir, 'task-self_model-value.json')
with open(self_con_out, 'w') as fp:
    json.dump(self_con, fp, indent=4)

Make event files:

In [6]:
for sub in subjects:
    # Create subject directory
    out_dir = opj(model_dir, sub, 'func')
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    print('Saving model specification to: %s' % out_dir)

    # Search for event files
    sub_dir = opj(data_dir, sub, 'func')
    event_files = glob.glob(opj(sub_dir, '*task-self*events.tsv'))
    print('Found functional runs: %s' % event_files)

    # Iterate over event files
    for event_f in event_files:
        events = pd.read_csv(event_f, sep='\t')
        
        # Main effect of trial
        trials = events.copy()[['onset', 'duration', 'trial_type']]

        # Motor response
        motor_response = pd.DataFrame()
        motor_response['onset'] = events['onset'] + events['response_time']
        motor_response['duration'] = 0.
        motor_response['trial_type'] = ['motor-%s' % (d) for d in np.where(events.choice == 1, 'left', 'right')]
        motor_response.dropna(inplace=True)

        # Chosen value
        chosen_value = trials.copy()
        chosen_value['trial_type'] = 'self-chosen'
        chosen_value['amplitude'] = mean_center(events['self_value_c'])
        chosen_value.dropna(inplace=True)

        # Unchosen value
        unchosen_value = trials.copy()
        unchosen_value['trial_type'] = 'self-unchosen'
        unchosen_value['amplitude'] = np.where(events['choice'] == 1, events['self_value2'], events['self_value1'])
        unchosen_value['amplitude'] = mean_center(unchosen_value['amplitude'])
        unchosen_value.dropna(inplace=True)

        # Build model
        model = pd.concat([trials, motor_response, chosen_value, unchosen_value])
        model = model[['onset', 'duration', 'trial_type', 'amplitude']]
        model.sort_values(axis=0, by=['onset', 'trial_type'], inplace=True)
        model.reset_index(drop=True, inplace=True)

        # Save output
        out_f = os.path.basename(event_f).replace('_run', '_model-value_run')
        out_path = opj(out_dir, out_f)
        model.to_csv(out_path, sep='\t', index=None)

Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-self_run-02_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-self_run-01_events.tsv']


/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-04/func/sub-04_task-self_run-02_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-04/func/sub-04_task-self_run-01_events.tsv']
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-05/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-self_run-01_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-self_run-02_events.tsv']
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-06/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-06/func/sub-06_task-self_run-01_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-06/func/sub-06_task-self_run-02_events.tsv']
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-07/func
Found functional runs: ['/

## Test

Contrasts:

In [7]:
test_names = ['test', 'motor-left', 'motor-right', 
              'self-chosen', 'self-unchosen',
              'other-chosen', 'other-unchosen']
test_con = [
    ('self: chosen', 'T', test_names, [0, 0, 0, 1, 0, 0, 0]),
    ('self: unchosen', 'T', test_names, [0, 0, 0, 0, 1, 0, 0]),
    ('other: chosen', 'T', test_names, [0, 0, 0, 0, 0, 1, 0]),
    ('other: unchosen', 'T', test_names, [0, 0, 0, 0, 0, 0, 1]),
    ('self: chosen > unchosen', 'T', test_names, [0, 0, 0, 1, -1, 0, 0]),
    ('self: unchosen > chosen', 'T', test_names, [0, 0, 0, -1, 1, 0, 0]),
    ('other: chosen > unchosen', 'T', test_names, [0, 0, 0, 0, 0, 1, -1]),
    ('other: unchosen > chosen', 'T', test_names, [0, 0, 0, 0, 0, -1, 1]),
    ('left > right', 'T', test_names, [0, 1, -1, 0, 0, 0, 0]),
    ('right > left', 'T', test_names, [0, -1, 1, 0, 0, 0, 0]),
    ('task', 'T', test_names, [1, 0, 0, 0, 0, 0, 0]),
    ('rest', 'T', test_names, [-1, 0, 0, 0, 0, 0, 0])
]

test_con_out = opj(model_dir, 'task-test_model-value.json')
with open(test_con_out, 'w') as fp:
    json.dump(test_con, fp, indent=4)

Make event files:

In [8]:
for sub in subjects:
    # Create subject directory
    out_dir = opj(model_dir, sub, 'func')
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    print('Saving model specification to: %s' % out_dir)

    # Search for event files
    sub_dir = opj(data_dir, sub, 'func')
    event_files = glob.glob(opj(sub_dir, '*task-test*events.tsv'))
    print('Found functional runs: %s' % event_files)

    # Iterate over event files
    for event_f in event_files:
        print('Reading data from: %s' % event_f)
        events = pd.read_csv(event_f, sep='\t')

        # Main effect of trial
        trials = events.copy()[['onset', 'duration', 'trial_type']]
        trials['trial_type'] = 'test'

        # Motor response
        motor_response = pd.DataFrame()
        motor_response['onset'] = events['onset'] + events['response_time']
        motor_response['duration'] = 0.
        motor_response['trial_type'] = ['motor-%s' % (d) for d in np.where(events.choice == 1, 'left', 'right')]
        motor_response.dropna(inplace=True)

        ### Self-relevant value ###
        # Chosen
        self_chosen = trials.copy()
        self_chosen['trial_type'] = 'self-chosen'
        self_chosen['amplitude'] = mean_center(events['SelfValue_c'])
        self_chosen.dropna(inplace=True)

        # Unchosen
        self_unchosen = trials.copy()
        self_unchosen['trial_type'] = 'self-unchosen'
        self_unchosen['amplitude'] = np.where(events['choice'] == 1, events['SelfValue2'], events['SelfValue1'])
        self_unchosen['amplitude'] = mean_center(self_unchosen['amplitude'])
        self_unchosen.dropna(inplace=True)

        ### Other-relevant value ###
        # Chosen
        other_chosen = trials.copy()
        other_chosen['trial_type'] = 'other-chosen'
        other_chosen['amplitude'] = mean_center(events['Value_c'])
        other_chosen.dropna(inplace=True)

        # Unchosen
        other_unchosen = trials.copy()
        other_unchosen['trial_type'] = 'other-unchosen'
        other_unchosen['amplitude'] = np.where(events['choice'] == 1, events['Value2'], events['Value1'])
        other_unchosen['amplitude'] = mean_center(other_unchosen['amplitude'])
        other_unchosen.dropna(inplace=True)

        # Build model
        model = pd.concat([trials, motor_response, self_chosen, self_unchosen, other_chosen, other_unchosen])
        model = model[['onset', 'duration', 'trial_type', 'amplitude']]
        model.sort_values(axis=0, by=['onset', 'trial_type'], inplace=True)
        model.reset_index(drop=True, inplace=True)

        # # Save output
        out_f = os.path.basename(event_f).replace('_run', '_model-value_run')
        out_path = opj(out_dir, out_f)
        print('Saving event file to: %s' % out_path)
        model.to_csv(out_path, sep='\t', index=None)

Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-test_run-01_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-test_run-02_events.tsv']
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-test_run-01_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func/sub-03_task-test_model-value_run-01_events.tsv
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-test_run-02_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func/sub-03_task-test_model-value_run-02_events.tsv
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-04/func/sub-04_task-test_run-01_events.tsv', '/scratch/groups/hyo/OPUS/B

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func/sub-04_task-test_model-value_run-01_events.tsv
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-04/func/sub-04_task-test_run-02_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func/sub-04_task-test_model-value_run-02_events.tsv
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-05/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-test_run-01_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-test_run-02_events.tsv']
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-test_run-01_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-05/func/sub-05_task-test_model-value_run-01_events.tsv
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-test_run-02_events.t

## Training

Contrasts:

In [9]:
train_names = ['train', 'motor-left', 'motor-right', 
              'other-chosen', 'other-unchosen', 'PE']
train_con = [
    ('chosen', 'T', train_names, [0, 0, 0, 1, 0, 0]),
    ('unchosen', 'T', train_names, [0, 0, 0, 0, 1, 0]),
    ('chosen > unchosen', 'T', train_names, [0, 0, 0, 1, -1, 0]),
    ('unchosen > chosen', 'T', train_names, [0, 0, 0, -1, 1, 0]),
    ('PE', 'T', train_names, [0, 0, 0, 0, 0, 1]),
    ('left > right', 'T', train_names, [0, 1, -1, 0, 0, 0]),
    ('right > left', 'T', train_names, [0, -1, 1, 0, 0, 0]),
    ('task', 'T', train_names, [1, 0, 0, 0, 0, 0]),
    ('rest', 'T', train_names, [-1, 0, 0, 0, 0, 0])
]

train_con_out = opj(model_dir, 'task-train_model-value.json')
with open(train_con_out, 'w') as fp:
    json.dump(train_con, fp, indent=4)

Make event files:

In [10]:
for sub in subjects:
    # Create subject directory
    out_dir = opj(model_dir, sub, 'func')
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    print('Saving model specification to: %s' % out_dir)

    # Search for event files
    sub_dir = opj(data_dir, sub, 'func')
    event_files = glob.glob(opj(sub_dir, '*task-train*events.tsv'))
    print('Found functional runs: %s' % event_files)

    # Iterate over event files
    for event_f in event_files:
        print('Reading data from: %s' % event_f)
        events = pd.read_csv(event_f, sep='\t')

        # Main effect of trial
        trials = events.copy()[['onset', 'duration', 'trial_type']]
        trials['trial_type'] = 'train'

        # Motor response
        motor_response = pd.DataFrame()
        motor_response['onset'] = events['onset'] + events['response_time']
        motor_response['duration'] = 0.
        motor_response['trial_type'] = ['motor-%s' % (d) for d in np.where(events.choice == 1, 'left', 'right')]
        motor_response.dropna(inplace=True)

        # Chosen
        other_chosen = trials.copy()
        other_chosen['trial_type'] = 'other-chosen'
        other_chosen['amplitude'] = mean_center(events['Value_c'])
        other_chosen.dropna(inplace=True)

        # Unchosen
        other_unchosen = trials.copy()
        other_unchosen['trial_type'] = 'other-unchosen'
        other_unchosen['amplitude'] = np.where(events['choice'] == 1, events['Value2'], events['Value1'])
        other_unchosen['amplitude'] = mean_center(other_unchosen['amplitude'])
        other_unchosen.dropna(inplace=True)

        # Prediction error
        pe = trials.copy()
        pe['trial_type'] = 'PE'
        pe['amplitude'] = mean_center(events['PE'])
        pe.dropna(inplace=True)

        # Build model
        model = pd.concat([trials, motor_response, other_chosen, other_unchosen, pe])
        model = model[['onset', 'duration', 'trial_type', 'amplitude']]
        model.sort_values(axis=0, by=['onset', 'trial_type'], inplace=True)
        model.reset_index(drop=True, inplace=True)

        # # Save output
        out_f = os.path.basename(event_f).replace('_run', '_model-value_run')
        out_path = opj(out_dir, out_f)
        print('Saving event file to: %s' % out_path)
        model.to_csv(out_path, sep='\t', index=None)

Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-train_run-02_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-train_run-01_events.tsv']
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-train_run-02_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func/sub-03_task-train_model-value_run-02_events.tsv
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-03/func/sub-03_task-train_run-01_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-03/func/sub-03_task-train_model-value_run-01_events.tsv
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-04/func/sub-04_task-train_run-01_events.tsv', '/scratch/groups/hyo

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func/sub-04_task-train_model-value_run-01_events.tsv
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-04/func/sub-04_task-train_run-02_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-04/func/sub-04_task-train_model-value_run-02_events.tsv
Saving model specification to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-05/func
Found functional runs: ['/scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-train_run-01_events.tsv', '/scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-train_run-02_events.tsv']
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-train_run-01_events.tsv
Saving event file to: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model/sub-05/func/sub-05_task-train_model-value_run-01_events.tsv
Reading data from: /scratch/groups/hyo/OPUS/BIDS_data/sub-05/func/sub-05_task-train_run-02_

## Probe

Contrasts:

In [ ]:
# Condition names
probe_trials = ['trial-%02d' % i for i in range(1, 33)]
probe_con_names = ['check'] + probe_trials

# Contrast values
probe_con_margin = np.zeros((32, 1))
probe_con_diag = np.zeros((32, 32))
np.fill_diagonal(probe_con_diag, 1)
probe_con_vals = np.concatenate((probe_con_margin, probe_con_diag), axis=1)
probe_con_vals = probe_con_vals.tolist()

# Contrasts
probe_con = [(probe_trials[t], 'T', probe_con_names, probe_con_vals[t]) for t in range(32)]
probe_con_out = opj(model_dir, 'task-probe_model-beta.json')
with open(probe_con_out, 'w') as fp:
    json.dump(probe_con, fp, indent=4)

Make event files:

In [ ]:
for sub in subjects[1:]:
    # Create subject directory
    out_dir = opj(model_dir, sub, 'func')
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    print('Saving model specification to: %s' % out_dir)

    # Search for event files
    sub_dir = opj(data_dir, sub, 'func')
    event_files = glob.glob(opj(sub_dir, '*task-probe*events.tsv'))
    print('Found functional runs: %s' % event_files)

    # Iterate over event files
    for event_f in event_files:
        print('Reading data from: %s' % event_f)
        events = pd.read_csv(event_f, sep='\t')

        # Check
        check = events[events['trial_type'] == 'check'].copy()
        check = check[['onset', 'duration', 'trial_type']]

        # Trials
        trials = events[events['trial_type'] != 'check'].copy()
        trials['trial_type'] = probe_trials

        # Build model
        model = pd.concat([check, trials])
        model = model[['onset', 'duration', 'trial_type']]
        model.sort_values(axis=0, by=['onset', 'trial_type'], inplace=True)
        model.reset_index(drop=True, inplace=True)

        # # # Save output
        out_f = os.path.basename(event_f).replace('_run', '_model-beta_run')
        out_path = opj(out_dir, out_f)
        print('Saving event file to: %s' % out_path)
        model.to_csv(out_path, sep='\t', index=None)